In [1]:
import numpy as np
import matplotlib.pylab as plt
import seaborn as sb
from scipy.spatial.distance import cdist
from scipy.integrate import odeint
import warnings
import sys
warnings.simplefilter('error')


"""
This program simulated the shock 


State of code :
    - Odeint integration
    - Verlet integration
    
Autor: Hernan Salinas 
Date_update : 2/21/2021
"""


'\nThis program simulated the shock \n\n\nState of code :\n    - Odeint integration\n    - Verlet integration\n    \nAutor: Hernan Salinas \nDate_update : 2/21/2021\n'

- 
\begin{equation}
\frac{D \rho}{Dt} = \sum_{j=1}^N m_j (v_j-v_i)\cdot \nabla_i W_{ij}
\end{equation}

-
\begin{equation}
\frac{D v_i}{Dt} = \sum_{j=1}^N m_j \left( \frac{P_i}{\rho_i^2}+ \frac{P_j}{\rho_j^2}\right) \nabla_i W_{ij}
\end{equation}


